# Module version used

- torch 1.4.0
- numpy 1.18.1
- CPython 3.6.10
- IPython 7.12.0

- Runs on CPU or GPU (if available)

# Multilayer Perceptron with Dropout

<img src="imgs/MLP_dropout.png" alt="Drawing" style="width: 600px;"/>

## Imports

In [22]:
import time
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch


# When running on the CuDNN backend, two further options must be set:

#torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False

#WARNING

# Deterministic mode can have a performance impact, depending on your model. 
# This means that due to the deterministic nature of the model, the processing
# speed (i.e. processed batch items per second) can be lower than when the model is non-deterministic.

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Settings and Dataset

In [23]:
##########################
### SETTINGS
##########################

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
random_seed = 1
learning_rate = 0.1
num_epochs = 10
batch_size = 64
dropout_prob = 0.5     # percentage of neurons that are dropped (set inactive)

# Architecture
num_features = 784
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10


##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='/Users/kundan/MEGA/DatabaseLocal/', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='/Users/kundan/MEGA/DatabaseLocal/', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [24]:
##########################
### MODEL
##########################

class MultilayerPerceptron(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(MultilayerPerceptron, self).__init__()
        
        ### 1st hidden layer
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        # The following to lones are not necessary, 
        # but used here to demonstrate how to access the weights
        # and use a different weight initialization.
        # By default, PyTorch uses Xavier/Glorot initialization, which
        # should usually be preferred.
        self.linear_1.weight.detach().normal_(0.0, 0.1)
        self.linear_1.bias.detach().zero_()
        
        ### 2nd hidden layer
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2)
        self.linear_2.weight.detach().normal_(0.0, 0.1)
        self.linear_2.bias.detach().zero_()
        
        ### Output layer
        self.linear_out = torch.nn.Linear(num_hidden_2, num_classes)
        self.linear_out.weight.detach().normal_(0.0, 0.1)
        self.linear_out.bias.detach().zero_()
        
    def forward(self, x):
        out = self.linear_1(x)
        out = F.relu(out)
        out = F.dropout(out, p=dropout_prob, training=self.training)
        
        out = self.linear_2(out)
        out = F.relu(out)
        out = F.dropout(out, p=dropout_prob, training=self.training)
        
        logits = self.linear_out(out)
        probas = F.softmax(logits, dim=1)
        return logits, probas

    
torch.manual_seed(random_seed)
model = MultilayerPerceptron(num_features=num_features,
                             num_classes=num_classes)

model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [25]:
def compute_accuracy(net, data_loader):
    net.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 28*28).to(device)
            targets = targets.to(device)
            logits, probas = net(features)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float()/num_examples * 100
    

start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))


    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))

    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 000/938 | Cost: 3.2452
Epoch: 001/010 | Batch 050/938 | Cost: 1.4599
Epoch: 001/010 | Batch 100/938 | Cost: 0.9749
Epoch: 001/010 | Batch 150/938 | Cost: 0.7050
Epoch: 001/010 | Batch 200/938 | Cost: 0.8016
Epoch: 001/010 | Batch 250/938 | Cost: 0.4514
Epoch: 001/010 | Batch 300/938 | Cost: 0.6336
Epoch: 001/010 | Batch 350/938 | Cost: 0.5292
Epoch: 001/010 | Batch 400/938 | Cost: 0.3572
Epoch: 001/010 | Batch 450/938 | Cost: 0.5326
Epoch: 001/010 | Batch 500/938 | Cost: 0.8996
Epoch: 001/010 | Batch 550/938 | Cost: 0.3955
Epoch: 001/010 | Batch 600/938 | Cost: 0.4036
Epoch: 001/010 | Batch 650/938 | Cost: 0.7397
Epoch: 001/010 | Batch 700/938 | Cost: 0.4987
Epoch: 001/010 | Batch 750/938 | Cost: 0.7858
Epoch: 001/010 | Batch 800/938 | Cost: 0.4577
Epoch: 001/010 | Batch 850/938 | Cost: 0.2588
Epoch: 001/010 | Batch 900/938 | Cost: 0.3637
Epoch: 001/010 training accuracy: 92.95%
Time elapsed: 0.35 min
Epoch: 002/010 | Batch 000/938 | Cost: 0.3673
Epoch: 002/010 |

Epoch: 009/010 | Batch 750/938 | Cost: 0.2742
Epoch: 009/010 | Batch 800/938 | Cost: 0.3448
Epoch: 009/010 | Batch 850/938 | Cost: 0.1622
Epoch: 009/010 | Batch 900/938 | Cost: 0.1177
Epoch: 009/010 training accuracy: 97.44%
Time elapsed: 3.40 min
Epoch: 010/010 | Batch 000/938 | Cost: 0.1459
Epoch: 010/010 | Batch 050/938 | Cost: 0.1235
Epoch: 010/010 | Batch 100/938 | Cost: 0.1938
Epoch: 010/010 | Batch 150/938 | Cost: 0.3822
Epoch: 010/010 | Batch 200/938 | Cost: 0.3493
Epoch: 010/010 | Batch 250/938 | Cost: 0.0760
Epoch: 010/010 | Batch 300/938 | Cost: 0.1244
Epoch: 010/010 | Batch 350/938 | Cost: 0.2507
Epoch: 010/010 | Batch 400/938 | Cost: 0.2091
Epoch: 010/010 | Batch 450/938 | Cost: 0.0459
Epoch: 010/010 | Batch 500/938 | Cost: 0.2355
Epoch: 010/010 | Batch 550/938 | Cost: 0.1934
Epoch: 010/010 | Batch 600/938 | Cost: 0.3408
Epoch: 010/010 | Batch 650/938 | Cost: 0.2423
Epoch: 010/010 | Batch 700/938 | Cost: 0.2392
Epoch: 010/010 | Batch 750/938 | Cost: 0.1962
Epoch: 010/010 |

In [26]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 96.82%
